## This demo presents the implementation for the following stories:
- RSPY-530 - Implement staging rs client

In [ ]:
import getpass
import json
import os
import pprint
import sys
import time
from datetime import datetime
from typing import Any

import boto3
import botocore
import requests
from pystac import Collection, Extent, SpatialExtent, TemporalExtent

from rs_client.rs_client import RsClient
from rs_client.rs_workflows.new_staging import RsStagingClient
from rs_common.config import ECadipStation
from rs_common.logging import Logging

In [ ]:
import requests
import os
import pprint
# Init environment before running a demo notebook.
from resources.utils import *

pp = pprint.PrettyPrinter(indent=2, width=80, sort_dicts=False, compact=True)
session = requests.Session()
user = os.environ["JUPYTERHUB_USER"] if cluster_mode else os.environ["RSPY_HOST_USER"]
auxip_client, cadip_client, stac_client = init_demo(owner_id = user)
if os.getenv("RSPY_LOCAL_MODE") == "1":
    href = "http://rs-server-cadip:8000"    
    href_staging = "http://rs-server-staging:8000"
else:
    href = os.environ["RSPY_WEBSITE"]
    href_staging = "https://rsserverstaging.dev-rspy.esa-copernicus.eu"
    session.cookies.set ("session", os.environ["RSPY_OAUTH2_COOKIE"])

cadip_collection_id = "cadip_s1A"
adgs_collection_id = "aux_interval"


In [ ]:
TIMEOUT = 10
CATALOG_BUCKET = "rs-cluster-catalog"
RSPY_HOST_STAGING = os.environ["RSPY_HOST_STAGING"]

APIKEY_HEADER = "x-api-key"
COLLECTION_ID = "cadip_s1A"
STAC_OUTPUT_COLL_NAME = "cadip_s1A_staged"
APIKEY_VALUE = None  # "x-api-key" ### TODO: get apikey from frontend page
APIKEY_HEADERS: dict = {"headers": {APIKEY_HEADER: APIKEY_VALUE}} if APIKEY_VALUE else {}
user = os.getenv("RSPY_HOST_USER", default=getpass.getuser())
local_mode = os.getenv("RSPY_LOCAL_MODE") == "1"
rs_server_href = "" if local_mode else os.getenv("RSPY_WEBSITE")

In [ ]:
 # Get client instances from the generic client
generic_client = RsClient(
    rs_server_href,
    rs_server_api_key=APIKEY_VALUE,
    owner_id=user,
    logger=None,
)
stac_client = generic_client.get_stac_client()
cadip_station = ECadipStation.CADIP  # you can also have: INS, MPS, MTI, NSG, SGS
cadip_client = generic_client.get_cadip_client(cadip_station)
logger = Logging.default(__name__)

In [ ]:
# Create the output STAC collection if it doesn't alredy exists
logger.info(f"Creating a new collection {STAC_OUTPUT_COLL_NAME} in the STAC catalog...")
try:
    create_coll_response = stac_client.get_collection(STAC_OUTPUT_COLL_NAME)
    logger.info(
        f"Collection {STAC_OUTPUT_COLL_NAME} already exists -> staging process will use the existing one",
    )
except TypeError:
    create_coll_response = stac_client.add_collection(
        Collection(
            id=STAC_OUTPUT_COLL_NAME,
            description=None,  # rs-client will provide a default description for us
            extent=Extent(
                spatial=SpatialExtent(bboxes=[-180.0, -90.0, 180.0, 90.0]),
                temporal=TemporalExtent(
                    [datetime(2000, 1, 1), datetime(2030, 1, 1)],
                ),
            ),
        ),
    )
    logger.info(
        f"Resp status: {create_coll_response.status_code} | Message: {create_coll_response.reason}",
    )

In [ ]:
# Create a bucket to store staged data
PREFIX = "stream/"
s3_session = boto3.session.Session()
s3_client = s3_session.client(
    service_name="s3",
    aws_access_key_id=os.environ["S3_ACCESSKEY"],
    aws_secret_access_key=os.environ["S3_SECRETKEY"],
    endpoint_url=os.environ["S3_ENDPOINT"],
    region_name=os.environ["S3_REGION"],
)
try:
    s3_client.head_bucket(Bucket=CATALOG_BUCKET)
    logger.info(f"The bucket {CATALOG_BUCKET} already exists")
except botocore.client.ClientError as error:
    if int(error.response["Error"]["Code"]) == 404:
        try:
            s3_client.create_bucket(Bucket=CATALOG_BUCKET)
        except botocore.exceptions.ClientError as e:
            logger.info(f"Bucket CATALOG_BUCKET error: {e}")
            sys.exit(-1)
    else:
        logger.info("PANIC: Could not get bucket info. Exiting")
        sys.exit(-1)
# Delete all existing objects from rs-server-catalog
if PREFIX:
    response = s3_client.list_objects_v2(Bucket=CATALOG_BUCKET, Prefix=PREFIX)
    if response.get("Contents", None):
        for elem in response["Contents"]:
            logger.info(f"Deleting {elem['Key']}")
            s3_client.delete_object(Bucket=CATALOG_BUCKET, Key=elem["Key"])

# Apply a request to get information about sessions that you want to stage
session = requests.Session()
search_result = session.get(f"{os.getenv('RSPY_HOST_CADIP')}/cadip/collections/{COLLECTION_ID}/items").json()

In [ ]:
# Launch staging 
staging_client = RsStagingClient()
staging_client.run_staging(APIKEY_HEADERS, search_result, STAC_OUTPUT_COLL_NAME, TIMEOUT)